
## Application of Physics/Expertise on Spectral Clustering 

#### Cristian Dominguez Godoy - [LinkedIn](https://www.linkedin.com/in/cfdominguez13/)
#### PhD Student, Machine Learning Applications for Formation Evaluation Research Group
#### Hildebrand department of Petroleum and Geosystems Engineering,  Cockrell School of Engineering, The University of Texas at Austin

### Subsurface Machine Learning Course, The University of Texas at Austin
#### Hildebrand Department of Petroleum and Geosystems Engineering, Cockrell School of Engineering
#### Department of Geological Sciences, Jackson School of Geosciences
_____________________

Workflow supervision and review by:

#### Instructor: Prof. Michael Pyrcz, Ph.D., P.Eng., Associate Professor, The Univeristy of Texas at Austin
##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

#### Course TA: Ademide Mabadeje, Graduate Student, The University of Texas at Austin
##### [LinkedIn](https://www.linkedin.com/in/ademidemabadeje/)


### Executive Summary

* Current clustering methods are statistical models used to minimize a certain error, be it distance between points or other. Current clustering techniques do not have any knowledge regarding the physics of the measurements nor of physical model trends. This workflow will implement industry expertise to the classification system by using affinity matrices and spectral clustering. To demonstrate spectral clustering with applied physics and expertise, I used the petrophysical correlation between permeability and porosity in core data.

* I created synthetic data that perfectly followed the physical model and correlations of the features being looked at, this would require expertise of the data/topic. Then I designed the affinity matrix by analysing the proximity of the sample data(data you want to cluster) to the perfect synthetic data(skeleton class). When applied to the dataset mentioned above, I am able to select the ideal number of clusters based on the petrophysical knowledge of the samples and capture the correct correlation of the data.

* The data has to be **mutually exclusive** and **exhaustive**. Based on the results it is possible to implement physical knowledge and expertise to your dataset through customized affinity matrices.
    * A sample point cannot belong to 2 different clusters 
    * All sample points must belong to a cluster group.<br> 

* I recommend using this classification modelling technique if you have dispersed data where you can apply physical models to make connections between data points. 


### Import Packages
The following packages and libraries are used in this workflow

In [1]:
import numpy as np                        # ndarrys for gridded data
import pandas as pd                       # DataFrames for tabular data
import os                                 # set working directory, run executables
import matplotlib.pyplot as plt           # for plotting
import matplotlib
import copy                               # for deep copies
import math                               # for square root function
from matplotlib.colors import ListedColormap # custom color bar
from sklearn.preprocessing import MinMaxScaler # min/max normalization
from sklearn.cluster import KMeans        # k-means clustering
from sklearn.cluster import SpectralClustering # spectral clustering
from ipywidgets import interactive        # widgets and interactivity
from ipywidgets import widgets            # for Widgets                
from ipywidgets import Layout             # for layout modification of widgets
from ipywidgets import Label              # for labeling of widgets
from ipywidgets import VBox, HBox         # for layout of widgets
cmap = plt.cm.inferno                     # color map with tone and intensity change

import scipy                              # to use the scipy library from python for modeling and statistics
import seaborn as sns                     # for matrix scatter plots and visualize data
from scipy import linalg                  # for linear regression and calculate vectors/ Laplace
import matplotlib.gridspec as gridspec    # to adjust plot shapes
from matplotlib.ticker import FixedLocator, LinearLocator, MultipleLocator  # to add tickers to plots
from matplotlib.cm import ScalarMappable  #to add colorbar to plots

### Load Data
The following workflow applies the .xlsx file "Spectral_dataset.xlsx", the file contains 3 sheets called "spectral","spectral2" and "spectral_syn". Sheet 1 contains sample data from real coredata, sheet 2 contains sample data 
synthetically made based on core data, and sheet 3 contains the physics/expertise of what a classification should be, this initial classification is created with synthetic data that would perfectly fit the classification.

We will work with the following features:
* **Porosity** - fraction of rock void in units of volume/volume (fraction)
* **Permeability** - ability of a fluid to flow through the rock in milliDarcy
* **Permeability/Porosity ratio** - Petrophysical rock classification method

In [2]:
spectral = pd.read_excel(r"https://raw.githubusercontent.com/cfd383/Machine_Learning/master/Spectral_dataset.xlsx" , sheet_name = 'Spectral')
spectral2 = pd.read_excel(r"https://raw.githubusercontent.com/cfd383/Machine_Learning/master/Spectral_dataset.xlsx" , sheet_name = 'Spectral2')
spectral_syn = pd.read_excel(r"https://raw.githubusercontent.com/cfd383/Machine_Learning/master/Spectral_dataset.xlsx" , sheet_name = 'Spectral_syn')

## Application of Physics/Expertise on Spectral Clustering
#### Introduction
clustering of data has always been an important aspect of all industries in order to better understand feature properties and correlations. Often these clustering techniques use no physical correlations between properties and therefore fail to pick up the desired clustering trend. This workflow demonstrates that it is posssible to implement physics and expertise to the clustering model in order to correctly classify the data. The phyics knowledge was applied to an affinity matrix which would later be used in spectral clustering to give the final clustering selection. Below is a summary workflow.

##### Dashboard 1: set-up affinity matrix by classifying sample data based on industry expertise 
* Upload data
* Select dataset using slider. Once dataset is selected:
    * Apply expertise to identify how to classify sample data based on the given physics skeleton frame
        * Provide sliders for number of clusters
        * Provide radius of proximity to the cluster centroid for a sample data to be considered part of cluster
        * Provide slider for physics control of each active cluster (active clusters determined by number of clusters)
        * Provide live graphical representation of cluster selection and coverage
        * Show internal calculations graphically for understanding purposes 
            * by plotting Adjacency matrix, eigen values and eigen vectors
* Make it possible to extract affinity matrix/calculated classification for future calulculations if needed

##### Dashboard 2: Compare expertise to other classification methods (KNN and Kmeans)
* Provide selection of dataset using slider
* provide sliders for hyperparameter tuning of other classification methods
* plot classification based on expertise side by side to the other classification methods (KNN and Kmeans)

## Dashboard 1. set-up affinity matrix by classifying sample data based on industry expertise
This workflow sets up the modifiable widgets which will allow the user to apply their expertise knowledge to the petrophysical properties of this dataset. The workflow works for any dataset that uses porosity, permeability, and permeability/porosity with the headers: "Porosity","k", and "K/por" respectively.
* You can slect which dataset to use based on the two sets uploaded "spectral" and "spectral2"
* Will allow for "n_clusters" classifications to be adjusted
    * Each classification will be updated live on the charts to see in which class each sample data belongs to.
* The main purpose of the function is to calculate the adjacecy matrix (affinity matrix) of the sample data based on the desired classifications.
    * The physics for classification on this dataset is done in 1 dimensional space. (Rocks are classified based on the ratio of Permeability/Porosity).
    * After calculating the 1D adjacency matrix it is converted to 2 dimensions (Permeability and Porosity in the y and x axis respectively) for easier visual understanding.

**Important note:** All sample data must belong to a classification, by default if the conditions set by user don't cover all points (i.e. the radius of centroid is to small to encompass all sample data) then the function will force all unclassified sample data to belong to the next nearest user set classification. The non covered points will have a black outline and a black line connecting it to the skeleton(user defined) classification(shown in top left graph). Top right graph shows the 1D representation of the classification the radius of each classification is shown and also if the pointsare connected they will be linked by the same color line. If a point is inbtween multple classification radii then the sample point will belong to the closest classification. Again, if the sample point is not in range then it will automatically link to the closest class shwon with a black line.
**All classifications must have at least 1 sample data else the system will fail.**

In [3]:
import warnings; warnings.simplefilter('ignore')

color = ['blue','red','green','purple','orange','gold','magenta','cyan']
style = {'description_width': 'initial'}
l = widgets.Text(value='Spectral Clustering for petrophysics k/Por classification, Cristian Dominguez Godoy, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))
sample_data = widgets.IntSlider(min = 1, max = 2, value = 1, step = 1, description = 'Sample dataset',orientation='vertical',
                          layout=Layout(width='100px', height='200px'))
sample_data.style.handle_color = 'gray'
ncluster = widgets.IntSlider(min = 1, max = 8, value = 3, step = 1, description = 'nclusters',orientation='vertical',
                          layout=Layout(width='100px', height='200px'))
ncluster.style.handle_color = 'gray'

radius = widgets.FloatSlider(min=0.0, max = 0.5, value = 0.20, step = 0.01, description = 'radius',
                        orientation='vertical',layout=Layout(width='100px', height='200px'),continuous_update=False)
radius.style.handle_color = 'gray'
# changes the cluster skeleton (affinity matrix)   
x1 = widgets.FloatSlider(min=10, max = 40000.0, value = 300, step = 100.0, description = '$class_1$',orientation='vertical',
                         layout=Layout(width='70px', height='200px'),readout_format = '.0f',style=style,continuous_update=False)
x1.style.handle_color = color[0]

x2 = widgets.FloatSlider(min=10, max = 40000, value = 2000, step = 100.0, description = '$class_2$',orientation='vertical',
                         layout=Layout(width='70px', height='200px'),readout_format = '.0f',style=style,continuous_update=False)
x2.style.handle_color = color[1]

x3 = widgets.FloatSlider(min=10, max = 40000, value = 9500, step = 100.0, description = '$class_3$',orientation='vertical',
                         layout=Layout(width='70px', height='200px'),readout_format = '.0f',style=style,continuous_update=False)
x3.style.handle_color = color[2]

x4 = widgets.FloatSlider(min=10, max = 40000, value = 5000, step = 100.0, description = '$class_4$',orientation='vertical',
                         layout=Layout(width='70px', height='200px'),readout_format = '.0f',style=style,continuous_update=False)
x4.style.handle_color = color[3]

x5 = widgets.FloatSlider(min=10, max = 40000, value = 18000, step = 100.0, description = '$class_5$',orientation='vertical',
                         layout=Layout(width='70px', height='200px'),readout_format = '.0f',style=style,continuous_update=False)
x5.style.handle_color = color[4]

x6 = widgets.FloatSlider(min=10, max = 40000, value = 800, step = 100.0, description = '$class_6$',orientation='vertical',
                         layout=Layout(width='70px', height='200px'),readout_format = '.0f',style=style,continuous_update=False)
x6.style.handle_color = color[5]

x7 = widgets.FloatSlider(min=10, max = 40000, value = 200, step = 100.0, description = '$class_7$',orientation='vertical',
                         layout=Layout(width='70px', height='200px'),readout_format = '.0f',style=style,continuous_update=False)
x7.style.handle_color = color[6]

x8 = widgets.FloatSlider(min=10, max = 40000, value = 16500, step = 100.0, description = '$class_8$',orientation='vertical',
                         layout=Layout(width='70px', height='200px'),readout_format = '.0f',style=style,continuous_update=False)
x8.style.handle_color = color[7]

uipars = widgets.HBox([sample_data,ncluster,radius,x1,x2,x3,x4,x5,x6,x7,x8],) 
uik = widgets.VBox([l,uipars],)

left=0; bottom=0; right=2.0; top=5.5; wspace=0.15; hspace=0.35;

try:
    adjacency_data2
except NameError:
        pass
else:
    del adjacency_data2   
    
def affinity_spectral(sample_data,ncluster,radius,x1,x2,x3,x4,x5,x6,x7,x8,classes=spectral_syn):    #Function to normalize data based on widget values, make classifications and visualize
    #set global variables to use externally from the function
    global adjacency2,normdata,normdata1,normdata2,normclass,colors1,colors2,kcols1,kcols2,adjacency_data1,adjacency_data2,classes1,classes2
    #select dataset to work with
    if sample_data ==1:
        df = spectral.copy()
    elif sample_data ==2:
        df = spectral2.copy()
    
    # create list of the skeleton clusters and respective colors for looping. Also set plot height variables
    colorlist = ['blue','red','green','purple','orange','gold','magenta','cyan']
    colors=[]
    values2 =[x1,x2,x3,x4,x5,x6,x7,x8]
    values =[]
    heights = [2,2,3,2]
        
    # limits future calcualtion to only work for the number of clusters selected
    for i in range(1,ncluster+1):
        values.append(values2[i-1]) 
        colors.append(colorlist[i-1])
        if ((i %2 ==0)&(i>1)) : # adds heights to incoming plots depending on number of clusters
            heights.append(2)
    # set fram for plots
    figs, axs = plt.subplots(ncols=2, nrows=4+ ncluster//2, constrained_layout=True,figsize =(12,16+2*ncluster//2),
                         gridspec_kw={
                           'width_ratios': [2, 2],
                           'height_ratios': heights,
                       'wspace': 0.2,
                       'hspace': 0.2})
    # creating lists for modification of skeleton matrix (each cluster requires a column for perm and k/por)
    kcols =[]
    kpcols =[]
    # create list with the max and min values of the test dataset for normilization depending on skeleton values
    kvalues = []
    kpvalues = []
    pvalues = []
    kvalues.append(df["k"].max());kvalues.append(df["k"].min());
    kpvalues.append(df["K/por"].max());kpvalues.append(df["K/por"].min());
    pvalues.append(df["Porosity"].max());pvalues.append(df["Porosity"].min());
    pvalues.append(classes["Por"].max());pvalues.append(classes["Por"].min());
    # loop that allows for the modification of the skeleton matrix and also adds the min and max of each skeleton cluster for normilization later
    # important: if your test dataset has porosity >41% then you need to chage the "classes" dataframe and add addition rows that include higher porosity values the k and k/por will update automaticaly in this loop
    # loop below updates the classes data i.e the physics corrleation of the data used. It allows to add additional physics of the k/por feature depending on the number of clusters set by user.
    for i ,value in zip(range(1,ncluster+1),values):
        classes['k/por %d'%i] = value
        classes["k%d"%i]=classes['k/por %d'%i]*classes["Por"]
        kvalues.append(classes["k%d"%i].max());kvalues.append(classes["k%d"%i].min())
        kpvalues.append(classes['k/por %d'%i][0])
        kcols.append("k%d"%i);kpcols.append("k/por %d"%i);
    # identify the max and min values of all of the data (including the skeleton cluster data and the test data)
    kmax = np.log10(max(kvalues)); kmin = np.log10(min(kvalues));
    pmax = (max(pvalues)); pmin = (min(pvalues));
    kpmax = np.log10(max(kpvalues)); kpmin = np.log10(min(kpvalues));
    normdata = df.copy()
    normclass = classes.copy()
    # take the log10 of k and k/por then normalize the test sample data and the skeleton data 
    normdata["Porosity"]= ((normdata["Porosity"])-pmin)/(pmax-pmin);
    normdata["k"]= (np.log10(normdata["k"])-kmin)/(kmax-kmin);
    normdata["K/por"]= (np.log10(normdata["K/por"])-kpmin)/(kpmax-kpmin);
    
    normclass["Por"]= ((normclass["Por"])-pmin)/(pmax-pmin);
    for i in range(1,ncluster +1):
        normclass["k%d"%i]= (np.log10(normclass["k%d"%i])-kmin)/(kmax-kmin);
        normclass["k/por %d"%i]= (np.log10(normclass["k/por %d"%i])-kpmin)/(kpmax-kpmin);
    # removes empty 4th plot
    gs = axs[1, 1].get_gridspec()
    axs[1,1].remove()
    # specify the sample data
    X1 = normdata["K/por"]; X2 = normdata["K/por"] #  x and y are the same feature for 1D classification of data based on the feature of interest
    # make list of all classification values
    allclasses =[]
    for i in range(1,ncluster +1):
        allclasses.append(normclass["k/por %d"%i][0])
    # need n*n matrices
    adjacency = np.zeros([len(X1),len(X1)])   
    degree = np.zeros([len(X1),len(X1)])
    # populate the adjacency matrix based on the proximity to the skeleton point in the 1D space (k/por vs y plot)
    # picks minimum distance to cluster, if 2 clusters reach the same point based on given radius then point will go to closest cluster
    # if a point is not within a skeleton it will force it to join the nearest skeleton class
    disc_point =[]
    for j in range(0,len(X1)):
        mindist =[]
        mindist.append(1000)
        for i in range(len(allclasses)):
            distance = math.sqrt((allclasses[i]-X1[j])*(allclasses[i]-X1[j]))
            mindist.append(distance)
            if (distance <= radius)&(distance <= min(mindist)):
                adjacency[j,j] = i+1;
        if all(radius < dist for dist in mindist):
            min_value = min(mindist)
            min_index = mindist.index(min_value)
            adjacency[j,j] = min_index;
            disc_point.append(j)
    # calculate the diagonal from the sum of the rows in the adjacency matrix    
    degree = np.diag(adjacency.sum(axis=1))

    # calculate the Laplacian matrix
    Laplace = degree-adjacency
    
    # plot the k/por vs y graph showing how the points are connected based on the skeleton class location and shows radius of skeleton class coverage
    axs[0,1].scatter(X1,X2*0,c = 'red',alpha = 0.2,edgecolors = 'black')
    for j in range(0,len(X1)):
        for i in range(len(allclasses)):
            if adjacency[j,j] == i+1:
                x1 = [X1[j],allclasses[i]]; x2 = [X2[j],allclasses[i]]
                if j in disc_point:
                    axs[0,1].plot(x1,np.multiply(x2,0),c = "black",alpha = 0.5,lw=2)
                else:
                    axs[0,1].plot(x1,np.multiply(x2,0),c = colors[i],alpha = 1,lw=2)
    for color, (columnName, columnData) in zip(colors,normclass[kpcols].iteritems()):
            axs[0,1].scatter(columnData[0],columnData[0]*0,c = color,alpha = 1.0,edgecolors = 'black')
            cc =plt.Circle((columnData[0],columnData[0]*0),radius, fill = False,color = color)
            axs[0,1].add_patch(cc)
    for i, txt in enumerate(normdata.index):
        axs[0,1].annotate(txt, (normdata["K/por"][i],normdata["K/por"][i]*0),xytext = (normdata["K/por"][i],normdata["K/por"][i]/5*np.power(-1.02,i)-np.power(-1,i+1)*0.065),arrowprops={'width':0.001,"headwidth":0.001,"headlength":0.001})    
    axs[0,1].set_ylim(-0.6,0.6)
    axs[0,1].set_xlabel('Normalized K/por'); axs[0,1].set_ylabel(''); axs[0,1].set_title('Sample Data proximity to clusters defined by physics')

    # re-do calculation using the adjacency matrix for the 1D and convert it into the 2D adjacency matrix   
    X1 = normdata["Porosity"]; X2 = normdata["k"] #2 paired features
    # create n*n matrices
    adjacency2 = np.zeros([len(X1),len(X2)])   # declare the 2D matrices
    degree2 = np.zeros([len(X1),len(X2)])
    # create list of all the conversions (1 for each class) for looping purposes and to work on the current amount of classes only
    con1=[]; con2=[]; con3=[]; con4=[];con5=[]; con6=[]; con7=[]; con8=[]
    allcons = [con1,con2,con3,con4,con5,con6,con7,con8]
    #seperates the previous adjacency matrix into the specfified classes
    for j in range(0,len(X1)):
        for i in range(len(allcons)):
            if adjacency[j,j] == i+1:
                allcons[i].append(j)
    # remake the adjacejcy matrix with correct weighting for each class             
    for j in range(len(allcons)):
        for i in allcons[j]:
            for k in allcons[j]:
                if k != i:
                    adjacency2[k,i] =1; adjacency2[i,k] =1
    # creates the global variables to be extracted to other dashboard and other external uses. Allows for having affinity matrix for 
    #both sample datasets at the same time.
    if sample_data ==1:
        adjacency_data1 = copy.deepcopy(adjacency2)
        classes1 = copy.deepcopy(classes)
        normdata1 = copy.deepcopy(normdata)
        colors1 = copy.deepcopy(colors)
        kcols1 = copy.deepcopy(kcols)
    elif sample_data ==2:
        adjacency_data2 = copy.deepcopy(adjacency2)
        classes2 = copy.deepcopy(classes)
        normdata2 = copy.deepcopy(normdata)
        colors2 = copy.deepcopy(colors)
        kcols2 = copy.deepcopy(kcols)
    # plot the new 2D adjacency matrix  
    axs[2,0].imshow(adjacency2,cmap = plt.cm.binary);
    axs[2,0].set_xlim([-0.5,len(X1)-0.5]); axs[2,0].set_ylim([-0.5,len(X2)-0.5]);
    axs[2,0].xaxis.set_major_locator(plt.MultipleLocator(2))
    axs[2,0].yaxis.set_major_locator(plt.MultipleLocator(2))
    axs[2,0].xaxis.set_minor_locator(plt.MultipleLocator(1)); axs[2,0].yaxis.set_minor_locator(plt.MultipleLocator(1))
    axs[2,0].set_xticklabels(np.arange(-1,len(X1)+1,2));axs[2,0].set_yticklabels(np.arange(-1,len(X1)+1,2))
    axs[2,0].set_title('Adjacency Matrix for k vs por'); axs[2,0].set_xlabel('$X_1$'); axs[2,0].set_ylabel('$X_2$'); 
    for ix in range(0,len(X1)-1):
        axs[2,0].plot([.50+float(ix),0.5+float(ix)],[-0.5,len(X1)-0.5],color = 'black', alpha = 0.3)    # creates the bold lines inside  graph
        axs[2,0].plot([-0.5,len(X1)-0.5],[.50+float(ix),0.5+float(ix)],color = 'black', alpha = 0.3)    # creates the bold lines inside  graph
    figs.colorbar(ScalarMappable(norm=matplotlib.colors.Normalize(vmin=0, vmax=1, clip=False),cmap=plt.cm.binary), ax=axs[2,0])
     # calculate the diagonal from the sum of the rows in the adjacency matrix
    degree2 = np.diag(adjacency2.sum(axis=1))
    # calculate the Laplacian matrix
    L = degree2-adjacency2 
    
    # find the eigenvalues and eigenvectors
    vals, vecs = np.linalg.eig(L)
    # sort with respect to eigen values
    vecs = vecs[:,np.argsort(vals)]
    vals = vals[np.argsort(vals)]
    #remove complex number portion of the vectors to remove errors in calculations
    vecs = np.real(vecs)
    # do K means clustering based on the number of clusters provided initially and apply it to vectors
    kmeans = KMeans(n_clusters=ncluster)             # instantiate k-means clusters
    #set the initial starting vector depending if the number of clusters is set to 1 by user
    if ncluster == 1:
        init = 0
    else:
        init = 1
    kmeans.fit(vecs[:,init:ncluster])                   # fit k-means clusters
    clusters = kmeans.labels_ 
    clusters = clusters + 1 # to not get a class 0
    
    # show plot of skeleton clusters and connectivity of sample data based on skeleton cluster in normalized space
    # to change to regular space change X1 = df["Porosity"]; X2 = df["k"], normclass to classes, normdata to df and plot y axis in log scale
    for color, (columnName, columnData) in zip(colors,normclass[kcols].iteritems()):
        axs[0,0].scatter(normclass["Por"],columnData,c = color,alpha = 0.2,edgecolors = 'black', s=2 )
        axs[0,0].plot(normclass["Por"],columnData,c = color,alpha = 0.2)
    for color,j in zip(colors,range(len(allcons))):
            for i in allcons[j]:
                unnormalizey = df["Porosity"][i]*classes["k/por %d"%(j+1)][0] # unnormalize y axis to calculate correct distance to skeleton class(necessary because when normalising and taking log10 "K/por" distnaces are affected relative to K and por values)
                liney = (np.log10(unnormalizey)-kmin)/(kmax-kmin)
                x1 = [X1[i],X1[i]]; x2 = [X2[i],liney]
                # makes black circle outline and black line to skeleton for datapoints not given a class by user
                if i in disc_point:
                    axs[0,0].scatter(X1[i],X2[i],c = color,alpha = 1,edgecolors = "black")
                    axs[0,0].plot(x1,x2,c = "black",alpha = 0.5)
                else: 
                    axs[0,0].scatter(X1[i],X2[i],c = color,alpha = 1,edgecolors = color)
                    axs[0,0].plot(x1,x2,c = color,alpha = 0.5)
    for i, txt in enumerate(normdata.index):
        axs[0,0].annotate(txt, (normdata["Porosity"][i],normdata["k"][i]))
    axs[0,0].set_xlim([0,1]); axs[0,0].set_ylim([0,1.1]); 
    axs[0,0].set_xlabel('normalized Porosity'); axs[0,0].set_ylabel('normalized log Permeability'); axs[0,0].set_title('Physical model applied to sample data')
    # axs[0,0].set_yscale("log") # uncomment if followed instructions to change scale to unnormalized
    
    # The following code colormap is from Professor Michael Pyrcz (@GeostatsGuy), Interactive_Spectral_clustering.ipynb from [GeostatsGuy GitHub](https://github.com/GeostatsGuy/PythonNumericalDemos/Interactive_Spectral_Clustering.ipynb)
    # make a custom colormap
    my_colormap = plt.cm.get_cmap('RdBu_r', 256)
    newcolors = my_colormap(np.linspace(0, 1, 256))
    white = np.array([256/256, 256/256, 256/256, 1])
    newcolors[100:150, :] = white                          # mask all correlations less than abs(0.8)
    custom_cmap = ListedColormap(newcolors)
    
    #plots eigen vector matrix
    axs[2,1].imshow(vecs,cmap = custom_cmap);
    axs[2,1].set_title('Eigen Vectors'); axs[2,1].set_xlabel('Eigen Vectors'); axs[2,1].set_ylabel('Features, $X_i$');
    axs[2,1].xaxis.set_major_locator(plt.MultipleLocator(2)); axs[2,1].yaxis.set_major_locator(plt.MultipleLocator(2))
    axs[2,1].xaxis.set_minor_locator(plt.MultipleLocator(1)); axs[2,1].yaxis.set_minor_locator(plt.MultipleLocator(1))
    axs[2,1].set_xticklabels(np.arange(-1,len(X1)+1,2));axs[2,1].set_yticklabels(np.arange(-1,len(X1)+1,2))
    for ix in range(0,len(X1)-1):
        axs[2,1].plot([.50+float(ix),0.5+float(ix)],[-0.5,len(X1)-0.5],color = 'black')    # creates the bold lines inside vector graph
        axs[2,1].plot([-0.5,len(X1)-0.5],[.50+float(ix),0.5+float(ix)],color = 'grey', alpha = 0.2)    # creates the bold lines inside
    figs.colorbar(ScalarMappable(norm=matplotlib.colors.Normalize(vmin=-1, vmax=1, clip=False),cmap=custom_cmap), ax=axs[2,1])
     
    #Plots eigen values increasing order
    axs[3,0].grid(alpha = 0.2); axs[3,0].plot(vals, c = 'red'); 
    axs[3,0].set_title('Eigen Values'); axs[3,0].set_xlabel('Eigen Value Index'); axs[3,0].set_ylabel('Eigen Values');
    axs[3,0].xaxis.set_major_locator(plt.MultipleLocator(2));axs[3,0].xaxis.set_minor_locator(plt.MultipleLocator(1))
    
    # plots first Eigen vector onto feature space (x-y, porosity-permeability)
    axs[3,1].scatter(X1,X2,c = vecs[:,0],s = 60,alpha = 1.0,edgecolors = 'black',vmin = -1.0, vmax = 1.0, cmap = custom_cmap) 
    axs[3,1].set_xlabel('normalized porosity'); axs[3,1].set_ylabel('normalized log permeability'); axs[3,1].set_title('Sample Data and First Eigen Vector')
    figs.colorbar(ScalarMappable(norm=matplotlib.colors.Normalize(vmin=-1, vmax=1, clip=False),cmap=custom_cmap), ax=axs[3,1])
    # plots nth Eigen vector onto feature space (x-y, porosity-permeability) depending on number of clusters set by user
    for i, vec_no in zip(range(2,10),["Second","Third","Fourth","Fifth","Sixth","Seventh","Eighth"]):
        if (((i%2)==0) & (i<ncluster+1)): 
            axs[3+i//2,0].scatter(X1,X2,c = vecs[:,i-1],s = 60,alpha = 1.0,edgecolors = 'black',vmin = -1.0, vmax = 1.0, cmap = custom_cmap) 
            axs[3+i//2,0].set_xlabel('normalized porosity'); axs[3+i//2,0].set_ylabel('normalized log permeability'); axs[3+i//2,0].set_title('Sample Data and %s Eigen Vector'%vec_no)
            figs.colorbar(ScalarMappable(norm=matplotlib.colors.Normalize(vmin=-1, vmax=1, clip=False),cmap=custom_cmap), ax=axs[3+i//2,0])
            if i ==ncluster:
                axs[3+i//2,1].remove()
        elif (((i%2)!=0) & (i<ncluster+1)):   
            axs[3+i//2,1].scatter(X1,X2,c = vecs[:,i-1],s = 60,alpha = 1.0,edgecolors = 'black',vmin = -1.0, vmax = 1.0, cmap = custom_cmap) 
            axs[3+i//2,1].set_xlabel('normalized porosity'); axs[3+i//2,1].set_ylabel('normalized log permeability'); axs[3+i//2,1].set_title('Sample Data and %s Eigen Vector'%vec_no)
            figs.colorbar(ScalarMappable(norm=matplotlib.colors.Normalize(vmin=-1, vmax=1, clip=False),cmap=custom_cmap), ax=axs[3+i//2,1])   
            
    # plots the finlized classification based on the skeleton clusters assigned
    axs[1,0].scatter(df["Porosity"],df["k"],c = clusters,s = 60,alpha = 1.0,edgecolors = 'black',vmin = 0.5, vmax = 8.5, cmap = plt.cm.Dark2) 
    for i, txt in enumerate(normdata.index):
        axs[1,0].annotate(txt, (df["Porosity"][i],df["k"][i]), xytext = (df["Porosity"][i]+0.0003,df["k"][i]+df["k"][i]/6))
    axs[1,0].set_xlabel('Porosity'); axs[1,0].set_ylabel('Permeability'); axs[1,0].set_title('Sample Data Graph and Assigned Clusters')
    axs[1,0].set_yscale("log")
    figs.colorbar(ScalarMappable(norm=matplotlib.colors.Normalize(vmin=0.5, vmax=8.5, clip=False),cmap=plt.cm.Dark2), ax=axs[1,0]) 
      
    print("\033[1m" +"All sample data points must be grouped to one of the physics correlations"+ "\033[0m","(the curved horizontal lines generated from\nexpertise)"
          "\033[1m" +" and"+ "\033[0m","every physics correlation must have at least one sample data point. The top right graph shows a 1D\nrepresentation of the physics model. The circles show the radius each classification encompasses.\n"
          "If a sample point is within two overlapping circles, it will join the class which is closer. The graph on the top left"
          "\nshows a 2D representation of the connection between the sample data and the physics based classifications.")
# connect the function to make the samples and plot to the widgets    
interactive_plot = widgets.interactive_output(affinity_spectral, {'sample_data':sample_data,'ncluster':ncluster, 'radius':radius, 
                                                      'x1':x1, 'x2':x2, 'x3':x3,
                                                      'x4':x4, 'x5':x5, 'x6':x6,
                                                      'x7':x7, 'x8':x8})
interactive_plot.clear_output(wait = True)             # reduce flickering by delaying plot updating

### Results Dashboard 1

The results are shown below: the nclusters determines how many classes you want, the radius determines how close the sample data has to be to the skeleton class(the curves in chart 1 top left) to be part of that class. The X sliders determine the position of the classes (skeleton). The first chart shows how the skeleton clusters are visualized and also how the sample data is conected with respect to the skeleton clusters. Top right chart is the normalized k/por 1D plot showing how the sample data is conected with respect to the skeleton cluster in 1D (The skeleton cluster centroid is the same color as the slider representing it). The second top left chart shows the finilized classification of the sample data based on the selected skelton classifications. The remianing plots show the adjacency plot for k vs por, eigen vectors, eigen values, and the sample data covered for each eigen vector

In [4]:
display(uik, interactive_plot)                            # display the interactive plot

Output(outputs=({'output_type': 'stream', 'text': '\x1bAll sample data points must be grouped to one of the…

## Dashboard 2: Compare expertise to other classification methods (KNN and Kmeans)
This worflow is designed to collect the parameters and variables tuned and calculated above for each dataset. The workflow sets up modifiable widgets which will allow the user to select the dataset they worked with above and compare their classification from their expertise to a classification made by K-nearest neighbors and Kmeans clustering. The user will have the ability to tune the hyperparameters of the other two classification methods to play around and try to match the clasifications.

**Note** The more complex the classification on expertise (number of classification, location of skeleton, radius) the harder it will be to get the other classification methods to match.

In [5]:
import warnings; warnings.simplefilter('ignore')

color = ['blue','green','green']
style = {'description_width': 'initial'}
l = widgets.Text(value='Comparing affinity matrix spectral clustering to different Spectral Clustering classifications methods, Cristian Dominguez Godoy, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))
# changes the cluster skeleton (affinity matrix)   
sample_data = widgets.IntSlider(min = 1, max = 2, value = 1, step = 1, description = 'Sample dataset',orientation='vertical',
                          layout=Layout(width='100px', height='200px'))
sample_data.style.handle_color = 'gray'

neighbors_knn = widgets.IntSlider(min=1, max =max(len(spectral),len(spectral2)) , value = 10, step = 1, description = 'n_neighbors for KNN',orientation='vertical',
                         layout=Layout(width='150px', height='200px'),readout_format = '.0f',style=style,continuous_update=False)
neighbors_knn.style.handle_color = color[0]

gamma_value = widgets.IntSlider(min=1, max = 2000, value = 1, step = 100.0, description = 'gamma for rbf model',orientation='vertical',
                         layout=Layout(width='150px', height='200px'),readout_format = '.0f',style=style,continuous_update=False)
gamma_value.style.handle_color = color[2]

uipars = widgets.HBox([sample_data,neighbors_knn,gamma_value],) 
uik = widgets.VBox([l,uipars],)

left=0; bottom=0; right=2.0; top=5.5; wspace=0.15; hspace=0.35;
def compare_clustering(sample_data,neighbors_knn,gamma_value):    #Function to normalize data based on widget values, make classifications and visualize    
    # collects cluster number from above
    nclusters = ncluster.value
    #uses data extracted from global variable in dashboard 1 depending on the sample dataset selected by user below
    if sample_data ==1:
        df = spectral.copy()
        classesx=copy.deepcopy(classes1)
        normdatax = copy.deepcopy(normdata1)
        colorsx = copy.deepcopy(colors1)
        kcolsx = copy.deepcopy(kcols1)
        affinity_matrix = copy.deepcopy(adjacency_data1)
    elif sample_data ==2:
        try:
            adjacency_data2
        except NameError:
            print("There is no affinity matrix for this dataset. Make sure you created an affinity matrix by setting "
                  "'Sample dataset' = 2 above")
        else:
            df=spectral2.copy()
            classesx=copy.deepcopy(classes2)
            normdatax = copy.deepcopy(normdata2)
            colorsx = copy.deepcopy(colors2)
            kcolsx = copy.deepcopy(kcols2)
            affinity_matrix = copy.deepcopy(adjacency_data2)
    # checks if variable exists, then makes model for each clustering method tested
    try:
        affinity_matrix
    except NameError:
        pass
    else:
        model = SpectralClustering(random_state=42, n_clusters=nclusters, affinity='precomputed', assign_labels="kmeans").fit(affinity_matrix)
        model2 = SpectralClustering(random_state=42, n_clusters=nclusters, affinity='nearest_neighbors',n_neighbors =int(neighbors_knn),
                                           assign_labels="kmeans").fit(normdatax[["Porosity",'k']])
        model3 = SpectralClustering(random_state=42, n_clusters=nclusters, affinity='rbf',gamma = int(gamma_value),
                                           assign_labels="kmeans").fit(normdatax[["Porosity",'k']])
        # plots expertise classification from dashboard 1
        plt.subplot(131)
        pairplot = sns.scatterplot(x = df["Porosity"],y = df["k"],markers='.')
        for i, txt in enumerate(df.index):
            pairplot.annotate(txt, (df["Porosity"][i],df["k"][i]), xytext = (df["Porosity"][i]+0.0003,df["k"][i]+df["k"][i]/6))
        plt.scatter(df["Porosity"],df["k"],c = model.labels_+1,s = 60,alpha = 1.0,edgecolors = 'black',vmin = 0.5, vmax = 8.5, cmap = plt.cm.Dark2) 
        plt.xlabel('Porosity'); plt.ylabel('Permeability'); plt.title('Sample data clustered using affinity matrix')
        cbar = plt.colorbar()
        cbar.set_label('Cluster Index', rotation=270, labelpad=20)
        for color, (columnName, columnData) in zip(colorsx,classesx[kcolsx].iteritems()):
            plt.scatter(classesx["Por"],columnData,c = color,alpha = 0.2,edgecolors = 'black', s=2 )
            plt.plot(classesx["Por"],columnData,c = color,alpha = 0.2)
        plt.yscale("log")
        plt.xlim([0,0.45]); plt.ylim([0.01,50000])
        # plots KNN classification
        plt.subplot(132)
        pairplot = sns.scatterplot(x = df["Porosity"],y = df["k"],markers='.')
        for i, txt in enumerate(df.index):
            pairplot.annotate(txt, (df["Porosity"][i],df["k"][i]), xytext = (df["Porosity"][i]+0.0003,df["k"][i]+df["k"][i]/6))
        plt.scatter(df["Porosity"],df["k"],c = model2.labels_+1,s = 60,alpha = 1.0,edgecolors = 'black',vmin = 0.5, vmax = 8.5, cmap = plt.cm.Dark2)
        plt.xlabel('Porosity'); plt.ylabel('Permeability'); plt.title('Sample data clustered using\n %d nearest neighbors'%neighbors_knn)
        cbar = plt.colorbar()
        cbar.set_label('Cluster Index', rotation=270, labelpad=20)
        for color, (columnName, columnData) in zip(colorsx,classesx[kcolsx].iteritems()):
            plt.scatter(classesx["Por"],columnData,c = color,alpha = 0.2,edgecolors = 'black', s=2 )
            plt.plot(classesx["Por"],columnData,c = color,alpha = 0.2)
        plt.yscale("log")
        plt.xlim([0,0.45]); plt.ylim([0.01,50000])
        # plots Kmeans classification
        plt.subplot(133)
        pairplot = sns.scatterplot(x = df["Porosity"],y = df["k"],markers='.')
        for i, txt in enumerate(df.index):
            pairplot.annotate(txt, (df["Porosity"][i],df["k"][i]), xytext = (df["Porosity"][i]+0.0003,df["k"][i]+df["k"][i]/6))
        plt.scatter(df["Porosity"],df["k"],c = model3.labels_+1,s = 60,alpha = 1.0,edgecolors = 'black',vmin = 0.5, vmax = 8.5, cmap = plt.cm.Dark2)
        cbar = plt.colorbar()
        cbar.set_label('Cluster Index', rotation=270, labelpad=20)
        for color, (columnName, columnData) in zip(colorsx,classesx[kcolsx].iteritems()):
            plt.scatter(classesx["Por"],columnData,c = color,alpha = 0.2,edgecolors = 'black', s=2 )
            plt.plot(classesx["Por"],columnData,c = color,alpha = 0.2)
        plt.xlabel('Porosity'); plt.ylabel('Permeability'); plt.title('Sample data clustered using\n rbf with gamma value of %d'%gamma_value)
        plt.yscale("log")
        plt.xlim([0,0.45]); plt.ylim([0.01,50000])
        plt.subplots_adjust(left=0, bottom=0, right=2.2, top=1, wspace=0.2, hspace=0.2)
    
interactive_plot = widgets.interactive_output(compare_clustering, {"sample_data":sample_data,'neighbors_knn':neighbors_knn,'gamma_value':gamma_value})
interactive_plot.clear_output(wait = True)               # reduce flickering by delaying plot updating

### Results Dashboard 2
Graph on the left is the classification determined from dashboard 1 based on expertise classification. Middle graph shows the KNN classification of the same sample data given the same number of classifications chosen from expertise above. The graph on the right shows the the Kmeans classification of the same sample data given the same number of classifications chosen from expertise above.

In [6]:
display(uik, interactive_plot)                   # display the interactive plot

Output()

### Remarks

This workflow is designed to show how expertise/physics of the features in your dataset have a significant impact on the classification of the data. The workflow applies the physics of the dataset features and allows for industry expertise to assign better classifications based on said physics. The worflow allows the extraction of the affinity matrix/classification for future uses and calculation. The interactivity of Dashboard 1 is to properly classify your data in an easy manner. The interactivity of Dashboard 2 is to see how physics and expertise impact the classification compared to using statistical clustering models.
I hope that you can classify your own porosity - permeability data in a way that uses physics giving you better classifications than other built in classification methods.

I encourage you to reach out via [LinkedIn](https://www.linkedin.com/in/cfdominguez13/) if you need me to further explain, are having errors in the workflow, or would like to work together on a project. My research involves machine learning and applications to formation evaluation but it is not limited to petrophysics as my learned machine learning skills can be applied to all industries.